# Experments
Timings reported in paper

In [1]:
devtools::install_github("TobiasRoikjer/PtDAlgorithms")
library(ptdalgorithms)
set.seed(1234)

In [2]:
construct_rabbit_graph_R <- function(number_of_rabbits, flooding_rate_l, flooding_rate_r) {
    # We represent the vector as two integers, the number of rabbits on the left and right island
    state_vector_length <- 2
    graph <- create_graph(state_vector_length)
    initial_state <- c(number_of_rabbits, 0)
    # The initial state is the only starting state, with 100% starting probability
    add_edge(
      starting_vertex(graph),
      find_or_create_vertex(graph, initial_state),
      1
    )
    index <- 2
    # Iterate over all unvisited vertices
    while (index <= vertices_length(graph)) {
      vertex <- vertex_at(graph, index)
      state <- vertex$state
      if (state[1] > 0) {
        # Rabbit jump left to right
        child_state <- c(state[1] - 1, state[2] + 1)
        add_edge(
          vertex,
          find_or_create_vertex(graph, child_state),
          1
        )

        # Left island flooding
        child_state <- c(0, state[2])
        add_edge(
          vertex,
          find_or_create_vertex(graph, child_state),
          flooding_rate_l
        )
      }

      if (state[2] > 0) {
        # Rabbit jump right to left
        child_state <- c(state[1] + 1, state[2] - 1)
        add_edge(
          vertex,
          find_or_create_vertex(graph, child_state),
          1
        )
        # Right island flooding with rate of 4
        child_state <- c(state[1], 0)
        add_edge(
          vertex,
          find_or_create_vertex(graph, child_state),
          flooding_rate_r
        )
      }
      index <- index + 1
    }
    return(graph)
}

In [3]:
Rcpp::sourceCpp("./rabbit_construction_c.cpp")

In [13]:
RABBIT_NUMBERS <- c(10, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300,1400,1500)
df <- data.frame()

for (rabbits in RABBIT_NUMBERS) {
    start <- proc.time()[3]
    if (rabbits <= 500) {
    construct_rabbit_graph_R(rabbits, 2, 4)
    }
    end <- proc.time()[3]
    time_R <- end-start
    start <- proc.time()[3]
    
    graph <- construct_rabbit_graph(rabbits, 2, 4)
    
    end <- proc.time()[3]
    time_c = end-start
    df <- rbind(df, list(rabbits=rabbits, time_c=time_c, time_R=time_R, vertices=vertices_length(graph)))
}

df

rabbits,time_c,time_R,vertices
<dbl>,<dbl>,<dbl>,<int>
10,0.001,0.002,67
50,0.000,0.263,1327
100,0.003,0.262,5152
200,0.020,1.297,20302
300,0.039,5.867,45452
400,0.089,5.222,80602
500,0.228,8.179,125752
600,0.211,0.000,180902
700,0.352,0.000,246052


In [ ]:
RABBIT_NUMBERS <- c(10, 50, 100, 150, 200, 250)
df <- data.frame()

for (rabbits in RABBIT_NUMBERS) {
    graph <- construct_rabbit_graph(rabbits, 2, 4)
    matrix_time <- NA
    if (rabbits <= 100) {
        M <- graph_as_matrix(graph)
    
    start <- proc.time()[3]
   
        solve(M$SIM)
    end <- proc.time()[3]
        matrix_time <- end - start
    }
    
    start <- proc.time()[3]
    expectation(graph)
    end <- proc.time()[3]
    time = end-start
    df <- rbind(df, list(rabbits=rabbits, time=time, matrix_time=matrix_time,vertices=vertices_length(graph)))
}

df

In [7]:
RABBIT_NUMBERS <- c(10, 50, 100, 150, 200)
df <- data.frame()

for (rabbits in RABBIT_NUMBERS) {
    graph <- construct_rabbit_graph(rabbits, 2, 4)

      expectation(graph)
    start <- proc.time()[3]
                  for (i in 1:100) {
    expectation(graph)
                      }
    end <- proc.time()[3]
    time = end-start
    df <- rbind(df, list(rabbits=rabbits, time=time,vertices=vertices_length(graph)))
}

df

rabbits,time,vertices
<dbl>,<dbl>,<int>
10,0.000,67
50,0.013,1327
100,0.090,5152
150,0.324,11477
200,0.755,20302


In [11]:
RABBIT_NUMBERS <- c(10, 50, 100, 150, 200)
df <- data.frame()

for (rabbits in RABBIT_NUMBERS) {
    graph <- construct_rabbit_graph(rabbits, 2, 4)

    start <- proc.time()[3]
    ctx <- distribution_context(graph, 10000)
    
    while (distribution_context_state(ctx)$cdf < 0.99) {
        distribution_context_step(ctx)
    }
    end <- proc.time()[3]
    time = end-start
    df <- rbind(df, list(rabbits=rabbits, time=time,vertices=vertices_length(graph)))
}

df

rabbits,time,vertices
<dbl>,<dbl>,<int>
10,0.135,67
50,0.854,1327
100,2.945,5152
150,7.474,11477
200,30.452,20302
